# <center>Лабораторная работа №11-12
## <center>Линейная регрессия, Lasso и RF-регрессия в задаче по определению качества вина

![img](https://raw.githubusercontent.com/soolstafir/Machine-learning-course/master/img/wine_quality.jpg)<br>

In [2]:
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
# from sklearn.metrics.regression import mean_squared_error
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
from sklearn.ensemble import RandomForestRegressor

Будем работать с набором данных по качеству белого вина (репозиторий UCI).<br>
Загружаем данные.

In [3]:
url = 'https://raw.githubusercontent.com/soolstafir/Machine-learning-course/master/Labs/ml_lab-11-12_regression_wine/winequality-white.csv'
data = pd.read_csv(url, sep=';')

In [4]:
data.tail(7)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
4891,5.7,0.21,0.32,0.9,0.038,38.0,121.0,0.99074,3.24,0.46,10.6,6
4892,6.5,0.23,0.38,1.3,0.032,29.0,112.0,0.99298,3.29,0.54,9.7,5
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7
4897,6.0,0.21,0.38,0.8,0.020,22.0,98.0,0.98941,3.26,0.32,11.8,6


In [5]:
(data['quality'] == 9).value_counts()

False    4893
True        5
Name: quality, dtype: int64

In [10]:
x1 = (data['quality']).min()
(data['quality'] == x1).value_counts()

False    4878
True       20
Name: quality, dtype: int64

In [45]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


Отделите целевой признак, разделите обучающую выборку в отношении 7:3 (30% - под оставленную выборку, пусть random_state=17) и отмасштабируйте данные с помощью StandardScaler.

In [46]:
y = np.array(data["quality"])
X = np.array(data.iloc[:,[x for x in range(0, 11)]])

X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size = 0.7, random_state=17)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)

## Линейная регрессия

Обучите простую линейную регрессию.

In [47]:
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)

LinearRegression()

<span style="color:red; font-size:2em;">Задание 1</span>

**Каковы среднеквадратичные ошибки линейной регрессии на обучающей и отложенной выборках?**

In [48]:
from sklearn.metrics import mean_squared_error
train_pred = linreg.predict(X_train_scaled)
test_pred = linreg.predict(X_holdout_scaled)
print("Mean squared error (train): %.3f", mean_squared_error(y_train, train_pred))
print("Mean squared error (test): %.3f", mean_squared_error(y_holdout, test_pred))

Mean squared error (train): %.3f 0.5321779540981313
Mean squared error (test): %.3f 0.58351984586918


Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина (учтите, что большие по модулю отрицательные значения коэффициентов тоже говорят о сильном влиянии). Создайте для этого новый небольшой DataFrame.

<span style="color:red; font-size:2em;">Задание 2</span>

**Какой признак линейная регрессия считает наиболее сильно влияющим на качество вина?**

In [49]:
linreg_coef = pd.DataFrame(linreg.coef_, index=data.columns[:11])
# x = linreg_coef.sort_values(0, ascending=False)
# np.absolute(x)

x = np.abs(linreg_coef)
print(x.sort_values(0, ascending=False))

                             0
density               0.674179
residual sugar        0.560499
volatile acidity      0.192491
pH                    0.151993
alcohol               0.140476
fixed acidity         0.100363
sulphates             0.065973
total sulfur dioxide  0.033827
citric acid           0.013249
chlorides             0.011563
free sulfur dioxide   0.010237


## Lasso-регрессия

Обучите Lasso-регрессию с небольшим коэффициентом $\alpha = 0.01$ (слабая регуляризация). Пусть опять random_state=17.

In [50]:
lasso1 = Lasso(alpha=0.01, random_state=17)
lasso1.fit(X_train_scaled, y_train)

Lasso(alpha=0.01, random_state=17)

Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина. Какой признак "отвалился" первым, то есть наименее важен для объяснения целевого признака в модели Lasso?

In [51]:
lasso1_coef = pd.DataFrame(lasso1.coef_, index=data.columns[:11])
x1 = np.abs(lasso1_coef)
x1.sort_values(0, ascending=False)

,0
alcohol,0.343034
residual sugar,0.251695
density,0.202199
volatile acidity,0.189060
pH,0.063068
sulphates,0.032678
free sulfur dioxide,0.001119
fixed acidity,0.000000
citric acid,0.000000
chlorides,0.000000


Теперь определите лучшее значение $\alpha$ в процессе кросс-валидации 5-кратной кросс-валидации. Используйте LassoCV и random_state=17.

In [52]:
alphas = np.logspace(-6, 2, 200)
# logspace returns numbers spaced evenly on a log scale
#lasso cv линейная регрессия с l1-регуляризацией
lasso_cv = LassoCV(alphas=alphas, random_state=17)
lasso_cv.fit(X_train_scaled, y_train)

LassoCV(alphas=array([1.00000000e-06, 1.09698580e-06, 1.20337784e-06, 1.32008840e-06,
       1.44811823e-06, 1.58856513e-06, 1.74263339e-06, 1.91164408e-06,
       2.09704640e-06, 2.30043012e-06, 2.52353917e-06, 2.76828663e-06,
       3.03677112e-06, 3.33129479e-06, 3.65438307e-06, 4.00880633e-06,
       4.39760361e-06, 4.82410870e-06, 5.29197874e-06, 5.80522552e-06,
       6.36824994e-06, 6.98587975e-0...
       1.18953407e+01, 1.30490198e+01, 1.43145894e+01, 1.57029012e+01,
       1.72258597e+01, 1.88965234e+01, 2.07292178e+01, 2.27396575e+01,
       2.49450814e+01, 2.73644000e+01, 3.00183581e+01, 3.29297126e+01,
       3.61234270e+01, 3.96268864e+01, 4.34701316e+01, 4.76861170e+01,
       5.23109931e+01, 5.73844165e+01, 6.29498899e+01, 6.90551352e+01,
       7.57525026e+01, 8.30994195e+01, 9.11588830e+01, 1.00000000e+02]),
        random_state=17)

In [53]:
#alpha_ the amount of penalization chosen by cross validation
lasso_cv.alpha_

0.0014992684327860455

Выведите коэффициенты "лучшего" Lasso в порядке убывания влияния на качество вина.

<span style="color:red; font-size:2em;">Задание 3</span>

**Какой признак "обнулился первым" в настроенной модели LASSO?**

In [54]:
lasso_cv_coef = pd.DataFrame(lasso1.coef_, index=data.columns[:11])
lasso_cv_coef.sort_values(0, ascending=False)
# coef_ is a parameter vector

,0
alcohol,0.343034
residual sugar,0.251695
pH,0.063068
sulphates,0.032678
free sulfur dioxide,0.001119
fixed acidity,-0.000000
citric acid,0.000000
chlorides,-0.000000
total sulfur dioxide,0.000000
volatile acidity,-0.189060


Оцените среднеквадратичную ошибку модели на обучающей и тестовой выборках.

<span style="color:red; font-size:2em;">Задание 4</span>

**Каковы среднеквадратичные ошибки настроенной LASSO-регрессии на обучающей и отложенной выборках?**

In [55]:
train_pred = lasso_cv.predict(X_train_scaled)
test_pred = lasso_cv.predict(X_holdout_scaled)
# Predict using the linear model.
print("Mean squared error (train): %.3f", mean_squared_error(y_train, train_pred))
print("Mean squared error (test): %.3f", mean_squared_error(y_holdout, test_pred))

Mean squared error (train): %.3f 0.5324734177725563
Mean squared error (test): %.3f 0.5812632364411522


## Случайный лес

Обучите случайный лес с параметрами "из коробки", фиксируя только random_state=17.

In [56]:
forest = RandomForestRegressor(random_state=17)
forest.fit(X_train_scaled, y_train)
#fit - build a forest of trees
print(X_train_scaled.shape)
print(y_train.shape)

(1469, 11)
(1469,)


In [57]:
# forest = RandomForestRegressor(max_depth=10, random_state=100)
# forest.fit(X_train_scaled, y_train)
# print(X_train_scaled.shape, y_train.shape)

<span style="color:red; font-size:2em;">Задание 5</span>

**Каковы среднеквадратичные ошибки случайного леса на обучающей выборке, на кросс-валидации (cross_val_score с scoring='neg_mean_squared_error' и остальными параметрами по умолчанию) и на отложенной выборке?**

In [58]:
train_pred = forest.predict(X_train_scaled)
test_pred = forest.predict(X_holdout_scaled)
cross = abs(cross_val_score(forest, X_train_scaled, y_train, scoring='neg_mean_squared_error').mean())

print("Mean squared error (train): %.3f", mean_squared_error(y_train, train_pred))
print("Mean squared error (cv): %.3f", cross)
print("Mean squared error (test): %.3f", mean_squared_error(y_holdout, test_pred))

Mean squared error (train): %.3f 0.0618717494894486
Mean squared error (cv): %.3f 0.4518899380093334
Mean squared error (test): %.3f 0.4884576844561097


Настройте параметры min_samples_leaf и max_depth с помощью GridSearchCV и опять проверьте качество модели на кросс-валидации и на отложенной выборке.

In [59]:
from sklearn.ensemble import RandomForestClassifier
forest_params = {'max_depth': list(range(10, 25)), 
                 'min_samples_leaf': list(range(1, 8)),
                 'max_features': list(range(6,12))}
#random forest is a meta estimator that fits a number of decision tree classifiers on
#various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting

rfc = RandomForestClassifier()
locally_best_forest = GridSearchCV(rfc, forest_params)
locally_best_forest.fit(X_train_scaled, y_train)

#ищется лучший лесной наборчик в зависимости от параметров лесной глубины, листиков на деревьях,
#и максимального числа функций,которые следует учитывать при поиске лучшего ?сплита?

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'max_depth': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
                                       20, 21, 22, 23, 24],
                         'max_features': [6, 7, 8, 9, 10, 11],
                         'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7]})

In [60]:
locally_best_forest.best_params_, locally_best_forest.best_score_

#best_score is a mean cross-validated score 

({'max_depth': 20, 'max_features': 6, 'min_samples_leaf': 3},
 0.6071904529729981)

К сожалению, результаты  GridSearchCV не полностью воспроизводимы (могут отличаться на разных платформах даже при фиксировании *random_state*). Поэтому обучите лес с параметрами max_depth=19, max_features=7, и min_samples_leaf=1 (лучшие в моем случае).

<span style="color:red; font-size:2em;">Задание 6</span>

**Каковы среднеквадратичные ошибки настроенного случайного леса на обучающей выборке, на кросс-валидации (cross_val_score с scoring='neg_mean_squared_error') и на отложенной выборке?**

In [61]:
forest = RandomForestClassifier(max_depth=19, min_samples_leaf=1, max_features=7, random_state=17)
forest.fit(X_train_scaled, y_train)

RandomForestClassifier(max_depth=19, max_features=7, random_state=17)

In [62]:
train_pred = forest.predict(X_train_scaled)
test_pred = forest.predict(X_holdout_scaled)
print("Mean squared error (train): %.3f", mean_squared_error(y_train, train_pred))
print("Mean squared error (test): %.3f", mean_squared_error(y_holdout, test_pred))

Mean squared error (train): %.3f 0.0
Mean squared error (test): %.3f 0.5940507436570429


Оцените важность признаков с помощью случайного леса.

<span style="color:red; font-size:2em;">Задание 7</span>

**Какой признак оказался главным в настроенной модели случайного леса?**

In [65]:
rf_importance = pd.DataFrame(forest.feature_importances_, index=data.columns[:11])
rf_importance.sort_values(0, ascending=False)



,0
alcohol,0.135287
volatile acidity,0.102058
density,0.090710
free sulfur dioxide,0.089098
pH,0.088661
total sulfur dioxide,0.086722
residual sugar,0.086069
chlorides,0.084576
citric acid,0.080716
sulphates,0.079046


Сделайте выводы о качестве моделей и оценках влияния признаков на качество вина с помощью этих трех моделей.

In [66]:
# rf_importance1 = pd.DataFrame(forest.feature_importances_, index=data.columns[:20])
# rf_importance1.sort_values(0, ascending=False)